In [1]:
import keras
import random, os
from keras.layers import Activation, Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Dropout, Flatten
from keras.models import Model, Sequential
from keras import backend as K
from glob import glob
from PIL import Image
import numpy as np
import re
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

Using TensorFlow backend.


In [2]:
batch_size = 128
num_classes = 6
epochs = 500
img_rows, img_cols = 28, 28
label = ["circle", "square", "triangle", "horizontal", "vertical", "diagonal"]

In [3]:
def get_datasets_labels(path):
	files = os.listdir(path)
	labels = []
	sets = []
	for file in files:
		pic = Image.open(path + file)
		lb = file.replace(re.findall("\d+", file)[0], '').replace('.png', '')
		labels.append(label.index(lb))
		pic = np.array(pic)
		sets.append(pic)
	return sets, labels

In [4]:
def load_data():
    tr_path = './dataset/trains/'
    te_path = './dataset/tests/'
    va_path = './dataset/vals/'
    
    (tr_sets, tr_labels) = get_datasets_labels(tr_path)
    (te_sets, te_labels) = get_datasets_labels(te_path)
    (va_sets, va_labels) = get_datasets_labels(va_path)
    
    
    tr_sets = np.array(tr_sets)
    te_sets = np.array(te_sets)
    va_sets = np.array(va_sets)
    tr_labels = np.array(tr_labels)
    te_labels = np.array(te_labels)
    va_labels = np.array(va_labels)
    return tr_sets, tr_labels, te_sets, te_labels, va_sets, va_labels

In [5]:
(x_train, y_train, x_test, y_test, x_val, y_val) = load_data()
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
x_val = x_val.reshape(x_val.shape[0], img_rows, img_cols, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_val = x_val.astype('float32')
x_train /= 255.
x_test /= 255.
x_val /= 255.

In [6]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2), padding='same'))

model.add(Conv2D(32, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2), padding='same'))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2), padding='same'))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2), padding='same'))

model.add(Flatten())
model.add(Dense(128, activation='relu'))

#model.add(Dropout(0.25))
model.add(Dense(num_classes, activation='softmax'))

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 14, 14, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 7, 7, 32)          0         
__________

In [8]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [9]:
model.fit(x_train, y_train,
          batch_size=batch_size
,          epochs=epochs,
          verbose=1, validation_data=(x_val, y_val))

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

#SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))

Train on 900 samples, validate on 600 samples
Epoch 1/500
900/900 [==============================] - 1s 2ms/step - loss: 1.7938 - acc: 0.1644 - val_loss: 1.7916 - val_acc: 0.1667
Epoch 2/500
900/900 [==============================] - 0s 44us/step - loss: 1.7934 - acc: 0.1556 - val_loss: 1.7913 - val_acc: 0.1667
Epoch 3/500
900/900 [==============================] - 0s 44us/step - loss: 1.7901 - acc: 0.1878 - val_loss: 1.7881 - val_acc: 0.2917
Epoch 4/500
900/900 [==============================] - 0s 42us/step - loss: 1.7850 - acc: 0.2744 - val_loss: 1.7830 - val_acc: 0.1667
Epoch 5/500
900/900 [==============================] - 0s 42us/step - loss: 1.7777 - acc: 0.2156 - val_loss: 1.8075 - val_acc: 0.1667
Epoch 6/500
900/900 [==============================] - 0s 46us/step - loss: 1.7740 - acc: 0.2067 - val_loss: 1.7488 - val_acc: 0.2100
Epoch 7/500
900/900 [==============================] - 0s 41us/step - loss: 1.7817 - acc: 0.1911 - val_loss: 1.7760 - val_acc: 0.1667
Epoch 8/500
900/9

900/900 [==============================] - 0s 39us/step - loss: 0.5069 - acc: 0.8100 - val_loss: 0.6771 - val_acc: 0.7600
Epoch 62/500
900/900 [==============================] - 0s 38us/step - loss: 0.4450 - acc: 0.8533 - val_loss: 1.0858 - val_acc: 0.5983
Epoch 63/500
900/900 [==============================] - 0s 37us/step - loss: 0.5318 - acc: 0.8089 - val_loss: 0.9203 - val_acc: 0.6533
Epoch 64/500
900/900 [==============================] - 0s 38us/step - loss: 0.4877 - acc: 0.8211 - val_loss: 0.7436 - val_acc: 0.7600
Epoch 65/500
900/900 [==============================] - 0s 39us/step - loss: 0.4842 - acc: 0.8378 - val_loss: 0.8981 - val_acc: 0.6400
Epoch 66/500
900/900 [==============================] - 0s 39us/step - loss: 0.4540 - acc: 0.8256 - val_loss: 0.6234 - val_acc: 0.7683
Epoch 67/500
900/900 [==============================] - 0s 36us/step - loss: 0.4526 - acc: 0.8344 - val_loss: 0.4911 - val_acc: 0.8167
Epoch 68/500
900/900 [==============================] - 0s 37us/step

900/900 [==============================] - 0s 36us/step - loss: 0.1187 - acc: 0.9567 - val_loss: 0.4060 - val_acc: 0.8717
Epoch 122/500
900/900 [==============================] - 0s 39us/step - loss: 0.2326 - acc: 0.9167 - val_loss: 0.2721 - val_acc: 0.9117
Epoch 123/500
900/900 [==============================] - 0s 45us/step - loss: 0.0785 - acc: 0.9733 - val_loss: 3.0384 - val_acc: 0.5933
Epoch 124/500
900/900 [==============================] - 0s 40us/step - loss: 0.6973 - acc: 0.8900 - val_loss: 0.3079 - val_acc: 0.8967
Epoch 125/500
900/900 [==============================] - 0s 37us/step - loss: 0.0962 - acc: 0.9700 - val_loss: 0.6536 - val_acc: 0.8200
Epoch 126/500
900/900 [==============================] - 0s 43us/step - loss: 0.1247 - acc: 0.9622 - val_loss: 0.4296 - val_acc: 0.8583
Epoch 127/500
900/900 [==============================] - 0s 51us/step - loss: 0.1168 - acc: 0.9589 - val_loss: 0.3209 - val_acc: 0.9200
Epoch 128/500
900/900 [==============================] - 0s 39

900/900 [==============================] - 0s 39us/step - loss: 0.0217 - acc: 0.9967 - val_loss: 0.2808 - val_acc: 0.9283
Epoch 182/500
900/900 [==============================] - 0s 38us/step - loss: 0.0247 - acc: 0.9933 - val_loss: 0.2717 - val_acc: 0.9200
Epoch 183/500
900/900 [==============================] - 0s 43us/step - loss: 0.0302 - acc: 0.9944 - val_loss: 0.2866 - val_acc: 0.9400
Epoch 184/500
900/900 [==============================] - 0s 53us/step - loss: 0.0314 - acc: 0.9922 - val_loss: 0.5434 - val_acc: 0.9100
Epoch 185/500
900/900 [==============================] - 0s 38us/step - loss: 0.0540 - acc: 0.9822 - val_loss: 0.3499 - val_acc: 0.9033
Epoch 186/500
900/900 [==============================] - 0s 37us/step - loss: 0.0441 - acc: 0.9889 - val_loss: 0.2731 - val_acc: 0.9333
Epoch 187/500
900/900 [==============================] - 0s 38us/step - loss: 0.0144 - acc: 0.9978 - val_loss: 0.2805 - val_acc: 0.9267
Epoch 188/500
900/900 [==============================] - 0s 40

900/900 [==============================] - 0s 35us/step - loss: 0.0213 - acc: 0.9944 - val_loss: 0.2735 - val_acc: 0.9317
Epoch 242/500
900/900 [==============================] - 0s 37us/step - loss: 0.0199 - acc: 0.9956 - val_loss: 0.2618 - val_acc: 0.9317
Epoch 243/500
900/900 [==============================] - 0s 37us/step - loss: 0.0131 - acc: 0.9978 - val_loss: 0.2709 - val_acc: 0.9333
Epoch 244/500
900/900 [==============================] - 0s 40us/step - loss: 0.0151 - acc: 0.9978 - val_loss: 0.2840 - val_acc: 0.9367
Epoch 245/500
900/900 [==============================] - 0s 49us/step - loss: 0.0188 - acc: 0.9944 - val_loss: 0.2865 - val_acc: 0.9333
Epoch 246/500
900/900 [==============================] - 0s 37us/step - loss: 0.0107 - acc: 1.0000 - val_loss: 0.2846 - val_acc: 0.9267
Epoch 247/500
900/900 [==============================] - 0s 37us/step - loss: 0.0108 - acc: 0.9989 - val_loss: 0.2982 - val_acc: 0.9267
Epoch 248/500
900/900 [==============================] - 0s 41

900/900 [==============================] - 0s 36us/step - loss: 0.0067 - acc: 0.9978 - val_loss: 0.3214 - val_acc: 0.9267
Epoch 302/500
900/900 [==============================] - 0s 37us/step - loss: 0.1296 - acc: 0.9656 - val_loss: 0.7050 - val_acc: 0.8633
Epoch 303/500
900/900 [==============================] - 0s 37us/step - loss: 0.1286 - acc: 0.9611 - val_loss: 0.2699 - val_acc: 0.9317
Epoch 304/500
900/900 [==============================] - 0s 38us/step - loss: 0.0114 - acc: 1.0000 - val_loss: 0.2722 - val_acc: 0.9317
Epoch 305/500
900/900 [==============================] - 0s 37us/step - loss: 0.0087 - acc: 0.9989 - val_loss: 0.2601 - val_acc: 0.9367
Epoch 306/500
900/900 [==============================] - 0s 46us/step - loss: 0.0075 - acc: 1.0000 - val_loss: 0.2621 - val_acc: 0.9333
Epoch 307/500
900/900 [==============================] - 0s 60us/step - loss: 0.0076 - acc: 0.9989 - val_loss: 0.2603 - val_acc: 0.9417
Epoch 308/500
900/900 [==============================] - 0s 36

900/900 [==============================] - 0s 37us/step - loss: 0.0019 - acc: 0.9989 - val_loss: 0.3154 - val_acc: 0.9317
Epoch 362/500
900/900 [==============================] - 0s 39us/step - loss: 0.0045 - acc: 0.9989 - val_loss: 0.2905 - val_acc: 0.9317
Epoch 363/500
900/900 [==============================] - 0s 38us/step - loss: 0.0017 - acc: 1.0000 - val_loss: 0.2845 - val_acc: 0.9433
Epoch 364/500
900/900 [==============================] - 0s 41us/step - loss: 7.6603e-04 - acc: 1.0000 - val_loss: 0.2911 - val_acc: 0.9367
Epoch 365/500
900/900 [==============================] - 0s 40us/step - loss: 0.0025 - acc: 1.0000 - val_loss: 0.2857 - val_acc: 0.9417
Epoch 366/500
900/900 [==============================] - 0s 36us/step - loss: 0.0023 - acc: 1.0000 - val_loss: 0.3104 - val_acc: 0.9417
Epoch 367/500
900/900 [==============================] - 0s 39us/step - loss: 0.0013 - acc: 1.0000 - val_loss: 0.3116 - val_acc: 0.9350
Epoch 368/500
900/900 [==============================] - 0

Epoch 421/500
900/900 [==============================] - 0s 44us/step - loss: 0.0099 - acc: 0.9978 - val_loss: 0.3435 - val_acc: 0.9367
Epoch 422/500
900/900 [==============================] - 0s 39us/step - loss: 0.0029 - acc: 0.9989 - val_loss: 0.3541 - val_acc: 0.9383
Epoch 423/500
900/900 [==============================] - 0s 40us/step - loss: 9.6213e-04 - acc: 1.0000 - val_loss: 0.3218 - val_acc: 0.9400
Epoch 424/500
900/900 [==============================] - 0s 40us/step - loss: 6.2412e-04 - acc: 1.0000 - val_loss: 0.3298 - val_acc: 0.9417
Epoch 425/500
900/900 [==============================] - 0s 44us/step - loss: 4.7908e-04 - acc: 1.0000 - val_loss: 0.3512 - val_acc: 0.9433
Epoch 426/500
900/900 [==============================] - 0s 38us/step - loss: 9.4408e-04 - acc: 1.0000 - val_loss: 0.3486 - val_acc: 0.9433
Epoch 427/500
900/900 [==============================] - 0s 40us/step - loss: 0.0016 - acc: 1.0000 - val_loss: 0.3751 - val_acc: 0.9333
Epoch 428/500
900/900 [=========

900/900 [==============================] - 0s 41us/step - loss: 2.9674e-04 - acc: 1.0000 - val_loss: 0.3293 - val_acc: 0.9450
Epoch 481/500
900/900 [==============================] - 0s 37us/step - loss: 2.2700e-04 - acc: 1.0000 - val_loss: 0.3341 - val_acc: 0.9417
Epoch 482/500
900/900 [==============================] - 0s 41us/step - loss: 2.4652e-04 - acc: 1.0000 - val_loss: 0.3553 - val_acc: 0.9350
Epoch 483/500
900/900 [==============================] - 0s 49us/step - loss: 0.0037 - acc: 0.9989 - val_loss: 0.2839 - val_acc: 0.9500
Epoch 484/500
900/900 [==============================] - 0s 39us/step - loss: 0.0019 - acc: 1.0000 - val_loss: 0.3608 - val_acc: 0.9433
Epoch 485/500
900/900 [==============================] - 0s 40us/step - loss: 3.2288e-04 - acc: 1.0000 - val_loss: 0.3664 - val_acc: 0.9400
Epoch 486/500
900/900 [==============================] - 0s 44us/step - loss: 5.2851e-04 - acc: 1.0000 - val_loss: 0.3592 - val_acc: 0.9450
Epoch 487/500
900/900 [===================

In [41]:
tc1 = np.array(x_test[0])
tc1 = tc1.reshape(1, 28, 28, 1)


In [42]:
model.layers[0].input

<tf.Tensor 'conv2d_1_input:0' shape=(?, 28, 28, 1) dtype=float32>

In [52]:
get_3rd_layer_output = K.function([model.layers[0].input], [model.layers[14].output])
layer_output = get_3rd_layer_output([tc1])[0]

In [55]:
layer_output.reshape(128, 1)

array([[ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 2.2404609 ],
       [ 0.        ],
       [ 0.        ],
       [ 7.619645  ],
       [ 0.        ],
       [ 4.0451837 ],
       [ 0.        ],
       [ 7.2664046 ],
       [ 0.5672691 ],
       [ 0.        ],
       [ 0.        ],
       [ 0.3500786 ],
       [ 0.        ],
       [ 0.8630257 ],
       [ 3.495305  ],
       [ 0.        ],
       [ 2.9216363 ],
       [ 3.1483338 ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 1.8700871 ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 1.9926243 ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 6.423704  ],
       [ 0.9234437 ],
       [ 0.        ],
       [ 0.        ],
       [ 0.25842804],
       [ 3.6050286 ],
       [ 5.0188956 ],
       [ 9